In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd 
import torch as torch 
import torch.nn as nn
import torch.optim as optim
import HD
import dataset_utils
import torch.nn.functional as F
import torchvision.models as models
from pytorch_metric_learning import distances, losses, miners, reducers

In [2]:

def binarize_hard(x):
    return torch.where(x > 0, 1.0, -1.0)

In [42]:
class BinaryModel(nn.Module):
    def __init__(self, dim, D, num_classes, enc_type='RP', binary=True, device='cpu', kargs=None):
        super(BinaryModel, self).__init__()
        self.enc_type, self.binary = enc_type, binary	
        self.device = device
        if enc_type in ['RP', 'RP-COS']:
            self.rp_layer = nn.Linear(dim, D).to(device)
            #self.rp_layer = quant_nn.Linear(dim, D, bias=False, quant_desc_input=tensor_quant.QUANT_DESC_4BIT_PER_TENSOR)
            self.class_hvs = torch.zeros(num_classes, D).bool().to(device)
            self.class_hvs_nb = torch.zeros(num_classes, D).bool().to(device)
        else:
            pass
    #hard sigmoid    
    def weight_binarize(self, W):
       W = torch.where(W<-1,-1,W)
       W = torch.where(W>1,1,W)
       mask1 = (W >= -1) & (W < 0)
       W[mask1] = 2 * W[mask1] + W[mask1]*W[mask1]
       mask2 = (W >= 0) & (W < 1)
       W[mask2] = 2 * W[mask2] - W[mask2]*W[mask2]
       # W[W > 1] = 1
       return W
    #using Bi-Real Approximation     
    def activation_binarize(self,a):
       a = torch.where(a<-1,-1,a)
       a = torch.where(a>1,1,a)
       mask1 = (a >= -1) & (a < 0)
       a[mask1] = 2 * a[mask1] + a[mask1]*a[mask1]
       mask2 = (a >= -0) & (a < 1)
       a[mask2] = 2 * a[mask2] - a[mask2]*a[mask2]
       #a = torch.where((a >= -1) & (a < 0),2*a + torch.pow(a,2) )
       #a = torch.where((a >= 0) & (a < 1), 2*a- torch.pow(a,2))
    #    a [a < -1] = -1
    #    a [a > 1]   =  1
    #    a [(a >= -1) & (a < 0)] = 2*a[(a >= -1) & (a < 0)] + torch.pow(a [(a >= -1) & (a < 0)],2)
    #    a [(a >= 0) & (a < 1)] = 2*a[(a >= 0) & (a < 1)] - torch.pow(a [(a >= 0) & (a < 1)],2)
       return a

    def encoding(self, x):
        if self.enc_type == 'RP':
            #x = self.activation_binarize(x) 
            #need not binarize the inputs 
            #progressively binarize the inputs, after training the weights
            #add some print statements and check 
            #print("The value of weights, before binarization")
            print(self.rp_layer.weight.data)
            weights = self.rp_layer.weight.data.clone()
            weights_bin = self.weight_binarize(weights)
            self.rp_layer.weight.data = weights_bin.clone() 
            
            #binarize the input to 4 bits.
            
            x_bin = self.Quantization(x,4)
            out = self.rp_layer(x_bin)
            
            print("The value of weights, after binarization")
            print(self.rp_layer.weight.data)
        else:
                pass
        
        return self.activation_binarize(out) if self.binary else out
    
    #Forward Function
    def forward(self, x, embedding=False):
        out = self.encoding(x)
        if embedding:
            out = out
        else:
            out = self.similarity(class_hvs=binarize_hard(self.class_hvs), enc_hv=out)   
        return out
    
    def init_class(self, x_train, labels_train):
        out = self.encoding(x_train)
        for i in range(x_train.size()[0]):
            self.class_hvs[labels_train[i]] += out[i]

        self.class_hvs = binarize_hard(self.class_hvs)
        
    def HD_train_step(self, x_train, y_train, lr=1.0):
        shuffle_idx = torch.randperm(x_train.size()[0])
        x_train = x_train[shuffle_idx]
        train_labels = y_train[shuffle_idx]
        l= list(self.rp_layer.parameters())
        enc_hvs = binarize_hard(self.encoding(x_train))
        for i in range(enc_hvs.size()[0]):
            sims = self.similarity(self.class_hvs, enc_hvs[i].unsqueeze(dim=0))
            predict = torch.argmax(sims, dim=1)
            
            if predict != train_labels[i]:
                self.class_hvs_nb[predict] -= lr * enc_hvs[i]
                self.class_hvs_nb[train_labels[i]] += lr * enc_hvs[i]
            
            self.class_hvs.data = binarize_hard(self.class_hvs_nb)
    def Quantization(self,X, bits):
        alpha = torch.min(X)
        beta = torch.max(X)
        alpha_q = -2 **(bits -1)
        beta_q = 2**(bits-1) -1
        s = (beta - alpha)/(beta_q - alpha_q)
        z = int((beta * alpha_q - alpha * beta_q) / (beta - alpha))
        X_q = torch.round(1/s * X + z,decimals = 0)
        X_q = torch.clip(X_q, min = alpha_q, max = beta_q)
        return X_q

    def similarity(self, class_hvs, enc_hv):
	    return torch.matmul(enc_hv, class_hvs.t())/class_hvs.size()[1]




In [4]:
A = torch.tensor(np.array([[0.5,2,3],[4,5,6]]))
A = torch.where((A >= 0), 1, A)
print(A)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [33]:
import torch

A = torch.tensor([[0.5, 1.2, 0.8],
                  [0.1, 0.3, 0.6]])

A = torch.min(A)
A

tensor(0.1000)

In [6]:
nFeatures, nClasses, x_train, y_train, x_test, y_test, train_loader, test_loader\
        = dataset_utils.load_dataset('mnist', 256, "cpu")


In [43]:

model = BinaryModel(dim = nFeatures, D=1000, num_classes=nClasses, enc_type='RP', device="cpu")

In [44]:
lr = 0.001
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0.0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0) 

def HD_test(model, x_test, y_test):
    out = model(x_test, embedding=False)
    preds = torch.argmax(out, dim=-1)

    acc = torch.mean((preds==y_test).float())	
    return acc

def get_Hamming_margin(model, x_test, y_test=None):
	def Hamming_distance(a, b):
		D = a.size()[1]
		return (D - a @ b.T)/2

	# Compute mean Hamming distance between class HVS
	class_hvs = binarize_hard(model.class_hvs.data)
	class_Hamming_distance = Hamming_distance(class_hvs, class_hvs)
	mean_class_Hamming_distance = torch.mean(class_Hamming_distance).item()
	

	# Compute test samples' Hamming distance
	test_enc_hvs = binarize_hard(model(x_test, True)) 
	test_Hamming_dist = Hamming_distance(test_enc_hvs, class_hvs)

	sorted_test_Hamming_distance, _ = torch.sort(test_Hamming_dist, dim=-1, descending=False)
	test_enc_hvs_Hamming_margin = (sorted_test_Hamming_distance[:,1:]-sorted_test_Hamming_distance[:,0].unsqueeze(dim=1)).mean(dim=1).cpu()
	mean_test_Hamming_margin = torch.mean(test_enc_hvs_Hamming_margin).item()

	res_dict = {
		"avg_class_Hamming_dist": mean_class_Hamming_distance,
		"avg_test_Hamming_margin": mean_test_Hamming_margin
	}
	return mean_test_Hamming_margin

In [45]:

num_metric_epochs = 2
device = "cpu"
accuracies = []
accuracies.append(HD_test(model, x_test, y_test).item())
margins = []
margins.append(get_Hamming_margin(model, x_test, y_test))

tensor([[ 0.0149, -0.0243, -0.0272,  ..., -0.0012,  0.0287,  0.0322],
        [ 0.0237, -0.0230, -0.0290,  ..., -0.0345,  0.0066, -0.0346],
        [ 0.0102, -0.0235,  0.0035,  ...,  0.0350, -0.0101,  0.0110],
        ...,
        [-0.0026, -0.0240, -0.0022,  ..., -0.0305, -0.0018,  0.0075],
        [-0.0135, -0.0133, -0.0174,  ...,  0.0288,  0.0252,  0.0183],
        [-0.0224, -0.0060,  0.0228,  ...,  0.0018,  0.0024,  0.0037]])
The value of weights, after binarization
tensor([[ 0.0296, -0.0481, -0.0536,  ..., -0.0025,  0.0566,  0.0633],
        [ 0.0469, -0.0455, -0.0571,  ..., -0.0678,  0.0131, -0.0680],
        [ 0.0202, -0.0464,  0.0070,  ...,  0.0688, -0.0200,  0.0218],
        ...,
        [-0.0052, -0.0474, -0.0044,  ..., -0.0602, -0.0035,  0.0149],
        [-0.0269, -0.0265, -0.0345,  ...,  0.0568,  0.0498,  0.0363],
        [-0.0444, -0.0120,  0.0451,  ...,  0.0037,  0.0048,  0.0074]])
tensor([[ 0.0296, -0.0481, -0.0536,  ..., -0.0025,  0.0566,  0.0633],
        [ 0.0469, -0.

In [ ]:

model.class_hvs = nn.parameter.Parameter(data=model.class_hvs)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard", )

In [ ]:
### pytorch-metric-learning using Triplet margin loss ###
print("Triplet Loss")
for epoch_i in range(1, num_metric_epochs + 1):
    HD.metric_train(model, loss_func, mining_func, device, train_loader, optimizer, epoch_i)
    accuracies.append(HD_test(model, x_test, y_test).item())
    margins.append(get_Hamming_margin(model, x_test, y_test))


In [ ]:
weights_updated = torch.zeros(model.rp_layer.weight.size())
weights_updated = model.rp_layer.weight.data.clone()    
weights_updated = binarize_hard(weights_updated)
#make the binarize function bipolar
model.rp_layer.weight.data  = weights_updated.clone()
print("Trained Weights")
print(model.rp_layer.weight.data)

In [ ]:
HD_lr = 0.05

In [ ]:
num_HD_epoch = 20
for epoch_i in range(1, num_HD_epoch+1):
    model.HD_train_step(x_train, y_train, lr=HD_lr)
    accuracies.append(HD_test(model, x_test, y_test).item())
    margins.append(get_Hamming_margin(model, x_test, y_test))


In [ ]:
plt.plot(accuracies, label="Accuracy", c = "blue")
plt.xlabel("iterations")
plt.ylabel("accuracy")
plt.title("Accuracy of 2 Levels, 1000 Dimensions Dataset:mnist")
plt.legend()
plt.show()
print("Maximum Accuracy:{}".format(max(accuracies)))
torch.save(model, './model/model.pth')

In [ ]:
print(model.rp_layer.weight.data)